In [ ]:
# STEP 0: SETUP AND IMPORTS
# This cell now imports your functions directly from ai_core.py

import os
import json
import getpass
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, Markdown
from PIL import Image

from ai_core import (
    get_identified_object,
    generate_spark_content,
    get_normalized_stem_skills,
    get_pathfinder_guidance,
    get_ai_tutor_response
)

load_dotenv()
try:
    api_key = os.getenv("GEMINI_API_KEY")
    if not api_key:
        api_key = getpass.getpass('GEMINI_API_KEY not found. Please enter it here: ')
    
    print(" Functions imported and ready to use.")
    if not os.getenv("GEMINI_API_KEY"):
         print("Note: API Key was entered manually.")

except Exception as e:
    print(f"An error occurred during setup: {e}")


uploaded_image_path = None
selected_grade_level = None
identified_object = None
spark_content = None
normalized_skills = None
user_skills_for_pathfinder = {}
tutor_chat_history = []

In [ ]:
# Step 1: Provide Inputs
# Select a grade level and upload an image file using the widgets below. After uploading, run the next cell to process the image.
grade_level_dropdown = widgets.Dropdown(
    options=['Elementary', 'Junior High School', 'Senior High School'],
    value='Junior High School',
    description='Grade Level:',
    disabled=False,
)

uploader = widgets.FileUpload(
    accept='image/*', 
    multiple=False,
    description='Upload Image'
)

print("Please select a grade level and upload an image:")

display(grade_level_dropdown, uploader)

In [ ]:
# Step 2: Process Inputs and Identify Object
# This cell takes the uploaded file, saves it temporarily, and runs `get_identified_object`.

if uploader.value:
    # --- Process the uploaded file ---
    uploaded_file_info = uploader.value[0]
    file_content = uploaded_file_info['content']
    file_name = uploaded_file_info['name']
    
    # Save the file temporarily to be used by the functions
    with open(file_name, 'wb') as f:
        f.write(file_content)
    
    uploaded_image_path = file_name
    selected_grade_level = grade_level_dropdown.value
    
    print(f"✅ Grade Level set to: {selected_grade_level}")
    print(f"✅ Image '{uploaded_image_path}' ready for processing.")
    
    # --- Display the image ---
    display(Image.open(uploaded_image_path))

    # --- Run Identification ---
    print("\n🔬 Identifying object in the image...")
    identified_object = get_identified_object(uploaded_image_path)
    
    if identified_object:
        display(Markdown("### 👁️ Object Identification Result:"))
        display(Markdown(f"**Object Label:** `{identified_object.get('object_label')}`"))
        display(Markdown(f"**Category Hint:** `{identified_object.get('category_hint')}`"))
    else:
        print("❌ Could not identify the object.")
else:
    print("⚠️ Please upload a file in the cell above and re-run this cell.")

In [ ]:
# Step 3: Generate "Spark" Educational Content
# This cell runs `generate_spark_content` using the identified object and selected grade level.

if identified_object and selected_grade_level:
    print("✨ Generating 'Spark' content...")
    spark_content = generate_spark_content(identified_object, selected_grade_level)
    
    if spark_content:
        display(Markdown("---"))
        display(Markdown("### 💡 Quick Facts"))
        display(Markdown(spark_content.get('quick_facts', '')))
        
        display(Markdown("### 🔬 STEM Concepts"))
        display(Markdown(spark_content.get('stem_concepts', '')))
        
        display(Markdown("### 🛠️ Hands-On Project"))
        display(Markdown(spark_content.get('hands_on_project', '')))
        display(Markdown("---"))
    else:
        print("❌ Could not generate Spark content.")
else:
    print("⚠️ Please run the previous steps first.")

In [ ]:
# Step 4: Extract STEM Skills and Test Pathfinder
# 1.  Runs `get_normalized_stem_skills` on the content from Step 3.
# 2.  Runs `get_pathfinder_guidance` using the extracted skills.

import random

if spark_content and selected_grade_level:
    # --- Part 1: Skill Extraction ---
    print("🌿 Extracting and normalizing STEM skills...")
    normalized_skills = get_normalized_stem_skills(spark_content)
    
    if normalized_skills:
        display(Markdown("### 🌳 Extracted Skills for Skill Tree:"))
        for skill in normalized_skills:
            display(Markdown(f"- **{skill.get('skill_name')}** (Category: *{skill.get('category')}*)"))
        
        # --- Part 2: Pathfinder Guidance ---
        # For testing, create a dummy skills dict with random mastery
        user_skills_for_pathfinder = {skill['skill_name']: random.randint(40, 95) for skill in normalized_skills}
        
        print("\n🧭 Generating Pathfinder guidance with the following skills profile:")
        print(json.dumps(user_skills_for_pathfinder, indent=2))

        pathfinder_guidance = get_pathfinder_guidance(user_skills_for_pathfinder, selected_grade_level)
        
        if pathfinder_guidance:
            display(Markdown("---"))
            display(Markdown(f"## 🧭 {pathfinder_guidance.get('title')}"))
            display(Markdown(f"_{pathfinder_guidance.get('summary')}_"))

            if pathfinder_guidance.get("recommendations"):
                 for rec in pathfinder_guidance.get("recommendations"):
                    display(Markdown(f"### Recommendation: {rec.get('name')}"))
                    display(Markdown(f"**Why?** {rec.get('why')}"))
                    if "whats_next" in rec:
                        display(Markdown(f"**What to Expect:** {rec.get('whats_next')}"))
                    if "career_paths" in rec:
                        display(Markdown(f"**Potential Careers:** {', '.join(rec.get('career_paths', []))}"))
                    if "local_spotlight" in rec:
                        display(Markdown(f"**Local Universities:** {', '.join(rec.get('local_spotlight', []))}"))
                    if "inspiration" in rec:
                        display(Markdown(f"**Inspiration:** {rec['inspiration'].get('name')}, {rec['inspiration'].get('description')}"))
            else:
                display(Markdown("No specific recommendations generated for this grade level, as expected."))
        else:
            print("❌ Could not generate Pathfinder guidance.")
    else:
        print("❌ Could not extract STEM skills.")
else:
    print("⚠️ Please run the previous steps first.")

In [ ]:
# Step 5: Test the AI Tutor (Interactive)
# Run this cell to ask a question. You can run it multiple times to simulate a conversation. The chat history is maintained between runs.

if selected_grade_level:
    object_context = identified_object.get('object_label') if identified_object else "the scanned object"
    
    user_question = input(f"Ask TuklasTutor about '{object_context}' or anything else: ")

    if user_question:
        # Add user's message to history
        tutor_chat_history.append({"role": "user", "parts": user_question})
        
        print("\n💬 Thinking...")
        ai_response = get_ai_tutor_response(
            user_question=user_question,
            grade_level=selected_grade_level,
            chat_history=tutor_chat_history,
            object_context=object_context
        )
        
        if ai_response:
            display(Markdown("---"))
            display(Markdown(f"**You:** {user_question}"))
            display(Markdown(f"**TuklasTutor:** {ai_response}"))
            display(Markdown("---"))
            
            # Add AI's response to history
            tutor_chat_history.append({"role": "model", "parts": ai_response})
        else:
            print("❌ Tutor did not provide a response.")
            # Remove user question from history if tutor fails
            tutor_chat_history.pop()
else:
    print("⚠️ Please run the previous steps first.")